In [2]:
!pip install pandas highway-env stable-baselines3 matplotlib tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import gymnasium as gym
import highway_env

from pandas import DataFrame
from stable_baselines3 import HerReplayBuffer, SAC
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [ ]:
from tqdm.auto import trange

# 환경 생성 - 차량 종류 다양화 활성화
env = gym.make('custom-parking-v0', render_mode=None, config={
    "vehicles_count": 5,
    "duration": 500,
    "vehicle_heading_offset": np.deg2rad(10),
    "vehicle_diversity": True,
    "test_vehicle_diversity": True,
    "controller_mode": "basic"
})

reset_features = []

max_steps = 500
round = 10000
model = SAC.load("sac_highway", env=env)

for reset_count in trange(round, desc="Episodes", unit="ep"):
    obs, _ = env.reset()

    vehicle = env.unwrapped.controlled_vehicles[0]
    features = env.unwrapped.extract_goal_features(vehicle)
    
    step = 0
    done = False
    while not done and step < max_steps:
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        frame = env.render()
        step += 1

    features["step"] = step
    features["done"] = done
    
    # done이 True일 때만 각도 점수 계산
    if done:
        angle_features = env.unwrapped.get_agent_angle_score(vehicle)
        features.update(angle_features)
    
    reset_features.append(features)
        
env.close()


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Episodes:   0%|          | 0/10000 [00:00<?, ?ep/s]

In [12]:
# 데이터프레임 생성
features_list = []
for idx, f in enumerate(reset_features):
    if f is not None:
        row = {"Reset": f"Reset {idx+1}"}
        for k, v in f.items():
            row[k] = v
        features_list.append(row)
if features_list:
    df_features = DataFrame(features_list)
else:
    print("저장할 feature 데이터가 없습니다.")
    df_features = DataFrame()

# csv로 저장
csv_path = "extract_goal_features_dataset.csv"
if not df_features.empty:
    df_features.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"extract_goal_features 데이터셋이 '{csv_path}'로 저장되었습니다.")
else:
    print("저장할 feature 데이터가 없습니다.")

extract_goal_features 데이터셋이 'extract_goal_features_dataset.csv'로 저장되었습니다.


In [ ]:
# 1만개만 따로 저장
if not df_features.empty:
    df_10000 = df_features.head(10000)
    csv_path_10000 = "extract_goal_features_dataset_10000.csv"
    df_10000.to_csv(csv_path_10000, index=False, encoding="utf-8-sig")
    print(f"extract_goal_features 1만개 데이터셋이 '{csv_path_10000}'로 저장되었습니다.")
else:
    print("저장할 feature 데이터가 없습니다.")
